In [ ]:
# Third-party imports.
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Tensorflow imports,
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.utils import to_categorical

# Preprocessing Data

In [100]:
# Unpacking data,
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Max normalisation,
X_train, X_test = X_train/np.max(X_train), X_test/np.max(X_test)

# One-hot encoding labels,
Y_train, Y_test = to_categorical(Y_train), to_categorical(Y_test)

# Adding channel dimension,
X_train = np.expand_dims(X_train, axis = -1)
X_test = np.expand_dims(X_test, axis = -1)

# Visualise Data

In [121]:
def show_image(i):
    """Function to display the samples of X_train, their shapes and their targets."""
    plt.figure(figsize=(3, 3))
    plt.imshow(np.squeeze(X_train[i]), cmap="gray")
    plt.axis("off")
    plt.show()
    
    print(f"Data Shape: {X_train[i].shape}")
    print(f"Target: {Y_train[i]}")

# Creating slider widget,
slider = widgets.IntSlider(min=0, max=len(X_train)-1, step=1, value=0, description="Index")

# Displaying widget,
widgets.interactive(show_image, i=slider)

interactive(children=(IntSlider(value=0, description='Index', max=59999), Output()), _dom_classes=('widget-int…

# Model Archecture & Helper Functions

In [122]:
class Net(keras.Model):
    """Class for the neural network architecture. Based on the LeNet-5 architecture."""

    def __init__(self):
        """Constructor method. Builds the layers and activations of the model."""
        super(Net, self).__init__()

        # First convolutional layer,
        self.pad1 = keras.layers.ZeroPadding2D(padding=2)  # Custom padding
        self.conv1 = keras.layers.Conv2D(filters=6, kernel_size=5, strides=1, activation="relu")
        self.pool1 = keras.layers.MaxPooling2D(pool_size=2, strides=2)

        # Second convolutional layer,
        self.conv2 = keras.layers.Conv2D(filters=16, kernel_size=5, strides=1, activation="relu")
        self.pool2 = keras.layers.MaxPooling2D(pool_size=2, strides=2)

        # Flattening data,
        self.flatten = keras.layers.Flatten()

        # Dense layers,
        self.fc1 = keras.layers.Dense(units=120, activation="relu")
        self.fc2 = keras.layers.Dense(units=84, activation="relu")
        self.fc3 = keras.layers.Dense(units=10, activation="softmax") 

    def call(self, x):
        """Forward pass of the neural network."""

        x = self.pad1(x)  
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

def compute_accuracy(X_test, Y_test):
    """Computes the accuracy of the model given a test dataset."""

    # Model predictions,
    Y_test_pred = model(X_test, training=False).numpy()

    # Comparing predictions,
    correct = 0
    for counter, Y_pred in enumerate(Y_test_pred):
        if np.argmax(Y_pred) == np.argmax(Y_test[counter]):
            correct += 1

    # Computing accuracy,
    accuracy = correct/(counter + 1)

    return accuracy

# View Model

In [123]:
# Creating model instance,
model = Net()

# Passing random tensor into model (TF uses channel first scheme),
model(tf.random.normal((1, 28, 28, 1))) 

# Print model summary
model.summary()

Model: "net_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ zero_padding2d_16               │ (1, 32, 32, 1)         │             0 │
│ (ZeroPadding2D)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (1, 28, 28, 6)         │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_31 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (1, 10, 10, 16)        │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (1, 400)               │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (1, 120)               │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (1, 84)                │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (1, 10)                │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

# Train Model

In [124]:
"""HYPERPARAMERS"""
EPOCHS = 1
LEARNING_RATE = 0.001
BATCH_SIZE = 128

# Creating dataloader,
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_dataset = train_dataset.shuffle(buffer_size=10000).batch(BATCH_SIZE)

# Defining loss function,
loss_fn = keras.losses.CategoricalCrossentropy()

# Creating optimiser,
optimiser = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# Training loop (epoch level),
for epoch in range(1, (EPOCHS + 1)):

    # Training loop (batch_level),
    for step, (X_batch, Y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:

            # Model forward pass,
            Y_pred = model(X_batch, training=True)

            # Computing loss,
            loss = loss_fn(Y_batch, Y_pred)

        # Computing gradients,
        gradients = tape.gradient(loss, model.trainable_variables)

        # Updating weights,
        optimiser.apply_gradients(zip(gradients, model.trainable_variables))


    print(f"Epoch: {epoch}, Training Loss: {loss}")

Epoch: 1, Training Loss: 0.09558213502168655


# Model Accuracy 

In [125]:
accuracy = compute_accuracy(X_test, Y_test)
print(f"Model Accuracy: {accuracy}")

Model Accuracy: 0.969
